# Data Engineer

## Bağlantı

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

adc.json  drive  sample_data


## Modul

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

## Dataset import

In [4]:
df_fake = pd.read_csv('/content/drive/Colab Notebooks/fake_news/Fake.csv')
df_true = pd.read_csv('/content/drive/Colab Notebooks/fake_news/True.csv')
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


# Data analysis

## ön işleme

In [5]:
df_fake['class']=0
df_true['class']=1

In [6]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [7]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23470,23460, -1):
    df_fake.drop([i], axis=0, inplace=True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21406, -1):
    df_true.drop([i], axis=0, inplace=True)

In [8]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)
df_true_manual_testing.to_csv('/content/drive/Colab Notebooks/fake_news/manual_testing.csv')

In [9]:
df_marge = pd.concat([df_fake, df_true], axis=0)
df_marge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [10]:
df = df_marge.drop(['title', 'subject', 'date'], axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [11]:
df = df.sample(frac=1)
df.head(10)

,text,class
20151,And this woman is going to try and keep up wit...,0
16227,REYKJAVIK (Reuters) - Angry over a string of p...,1
20619,Of course Obama the putz blames too little gov...,0
4726,WASHINGTON (Reuters) - Supporters of Obamacare...,1
3686,Her viral post won her the title of racist of...,0
11723,BRASILIA (Reuters) - The approval rating for B...,1
11579,"Sabo, an artist who has made a name for himsel...",0
6730,On April 23 Vice President Joe Biden cast his ...,0
5961,WASHINGTON (Reuters) - Britain’s Queen Elizabe...,1
4082,The math for the election is very simple: The ...,0


In [12]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?]', '', text)
    text = re.sub("\W"," ", text)
    text = re.sub('https?://\+S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [15]:
df['text'] = df['text'].apply(word_drop)

In [16]:
df.head(10)

,text,class
20151,and this woman is going to try and keep up wit...,0
16227,reykjavik reuters angry over a string of p...,1
20619,of course obama the putz blames too little gov...,0
4726,washington reuters supporters of obamacare...,1
3686,her viral post won her the title of racist of...,0
11723,brasilia reuters the approval rating for b...,1
11579,sabo an artist who has made a name for himsel...,0
6730,on april vice president joe biden cast his vo...,0
5961,washington reuters britain s queen elizabe...,1
4082,the math for the election is very simple the ...,0


In [17]:
x = df['text']
y = df['class']

In [18]:
from sklearn.model_selection import train_test_split

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= .25)                                                    

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [28]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
LR.score(xv_test, y_test)

0.9860071301247771

In [30]:
pred_LR = LR.predict(xv_test)

In [31]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5859
           1       0.98      0.99      0.99      5361

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### Decision Tree classification

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [35]:
DT.score(xv_test, y_test)

0.9952762923351158

In [37]:
pred_DT = DT.predict(xv_test)
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5859
           1       0.99      1.00      1.00      5361

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Gradient Boosting Classifier

In [38]:
from sklearn.ensemble import GradientBoostingClassifier

In [40]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [41]:
GBC.score(xv_test, y_test)

0.9963458110516934

In [42]:
pred_GBC = GBC.predict(xv_test)
print(classification_report(y_test, pred_GBC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5859
           1       0.99      1.00      1.00      5361

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Random Forest Classifier

In [43]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [45]:
RFC.score(xv_test, y_test)

0.9873440285204991

In [48]:
pred_RFC = RFC.predict(xv_test)
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5859
           1       0.99      0.99      0.99      5361

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### Manual Testing

In [64]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n ==1:
          return "Not A Fake News"
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test= pd.DataFrame(testing_news)
    new_def_test["text"]= new_def_test["text"].apply(word_drop)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLogistic Regression Prediction: {} \nDecision Tree classification Prediction: {} \nGradient Boosting Classifier Prediction: {} \nRandom Forest Classifier Prediction: {}".format(output_label(pred_LR),
                                                                                                                                                                                                      output_label(pred_DT),
                                                                                                                                                                                                      output_label(pred_GBC),
                                                                                                                                                                                                      output_label(pred_RFC),))

In [68]:
news = str(input())
manual_testing(news)

In the shadow of disused Soviet-era factories in Minsk, a street lined with eclectic bars, art galleries and yoga studios has become a haven from the vigilant eyes of the Belarussian authorities.  This place is like an island,  said Yegor, 21, who works at  popular bar Hooligan.  It s the street of freedom.  The government of President Alexander Lukashenko, who has ruled Belarus for the past 23 years and has boasted that he is  the last and only dictator in Europe  ,has little tolerance for any opposition. A powerful police force and feared state security  keep citizens in check. But police patrols are rare in Oktyabrskaya, partly due to its location on an out-of-the-way peninsula in a bend of the river Svislach. When the first restaurant opened there in 2012, few visitors came but now it ranks among the most fashionable quarters of Minsk.  Such is the growing popularity of that Oktyabrskaya that investors such as Belgazprombank, a subsidiary of state-owned Russian lender Gazprombank, 

### Fake News

In [ ]:
21st Century Wire says As 21WIRE predicted in its new year s look ahead, we have a new  hostage  crisis underway.Today, Iranian military forces report that two small riverine U.S. Navy boats were seized in Iranian waters, and are currently being held on Iran s Farsi Island in the Persian Gulf. A total of 10 U.S. Navy personnel, nine men and one woman, have been detained by Iranian authorities. NAVY STRAYED: U.S. Navy patrol boat in the Persian Gulf (Image Source: USNI)According to the Pentagon, the initial narrative is as follows: The sailors were on a training mission around noon ET when their boat experienced mechanical difficulty and drifted into Iranian-claimed waters and were detained by the Iranian Coast Guard, officials added. The story has since been slightly revised by White House spokesman Josh Earnest to follow this narrative:The 2 boats were traveling en route from Kuwait to Bahrain, when they were stopped and detained by the Iranians.According to USNI, search and rescue teams from the Harry S. Truman Strike group were scrambled to aid the crew but stopped short after the crew was taken by the Iranians. We have been in contact with Iran and have received assurances that the crew and the vessels will be returned promptly,  Pentagon spokesman Peter Cook told AP. According to Persian Gulf TV: Farsi Island is one of the Iranian islands in the Persian Gulf, Bushehr province of Iran. There is an IRGC Navy base on this Island. The island has an area of about 0.25 km  and is completely restricted to public, due to top secret governmental activities. According to NBC News, U.S. State Department is in touch with Tehran officials and the Iranians recognize that the U.S. Navy straying off course was a mistake, and that the sailors will be released  within hours.  WAR INC: CNN s Wolf Blitzer wasted no time in ramping-up talk of military tension with Israeli-financed neocon Senator Tom Cotton.Neocon StuntAlready, the U.S. media, including CNN and FOX, are running with the talking point that,  this could not have come at a worse time for President Obama right before tonight s State Of Union speech, when he s trying to prove to the American people that Iran is a country that can be trusted to implement the historic nuclear deal. This latest Naval  controversy  also comes days before the implementation phase of the Iran Nuclear Deal. To say this is a coincidence might be naive.That said, could GOP and Israel-aligned members of the Pentagon or intelligence establishment have helped to engineer today s bizarre  mini-crisis  in order to help weaken U.S.-Iran relations, and by extension, Obama s controversial Iranian Nuclear Deal?This looks likely to be the case, as evidenced by the quick appearance of the Israel Lobby-sponsored, pro-war U.S. Senator Tom Cotton (R), placed by CNN to direct aggressive U.S. military talking points live on air as the story broke today. Cotton (photo, left) immediately called the event  hostile  and blamed Iran for the U.S. boat drifting into Iranian waters, and then blamed the  crisis  on President Obama, who he claims,  has emboldened the Iranian aggression. Cotton then goes on to tell a giant lie, on which his media handler, CNN s Wolf Blitzer, does not even blink, much less challenge Cotton s imaginary statement: The Iranians, who are largely responsible for killing our (American) soldiers in Iraq and Afghanistan. Cotton then went on to threaten Iran, saying that: These sailors and both vessels  need to be immediately released. If they are not released, then the Iran (nuclear) deal should not go forward, and military force will be on the table to retaliate for this act of aggression.  Cotton then proceeded to give a veiled (nuclear?) threat to Iran, saying that,  All (military) options should be on the table.  Would Washington s top operatives go so far as to engineer or exacerbate an international crisis such as this   by dispatching the boats off course, knowing full-well that Iran would not harm U.S. personnel, but using the incident to injure a lame duck? The answer is  yes , and they ve done it before In 1979, 52 American diplomats and citizens were held hostage for 444 days in what became known as the Iranian Hostage Crisis, which just happened to take place during a US presidential election cycle, with then president Jimmy Carter (D) running for reelection against challenger Ronald Reagan (R). The crisis, including a horrific failed rescue attempt, was used against Carter in the media. According to reports at the time Reagan campaign operatives had managed to do a backdoor deal with the elements of the new Iranian regime to  hold-off  releasing the hostages until after the election. In the end, Reagan won and took credit for that  victory .Not surprisingly, at the end of his prearranged CNN segment, Cotton invoked the  feckless foreign policy  of Jimmy Carter which  caused  the 1979 Iran Hostage Crisis, and compared that to President Obama s current policy.Of all the U.S. Officials CNN could have brought in on  short notice , they chose Senator Tom Cotton, the most hawkish and closest aligned to Israel.

### Not A Fake News

In [ ]:
In the shadow of disused Soviet-era factories in Minsk, a street lined with eclectic bars, art galleries and yoga studios has become a haven from the vigilant eyes of the Belarussian authorities.  This place is like an island,  said Yegor, 21, who works at  popular bar Hooligan.  It s the street of freedom.  The government of President Alexander Lukashenko, who has ruled Belarus for the past 23 years and has boasted that he is  the last and only dictator in Europe  ,has little tolerance for any opposition. A powerful police force and feared state security  keep citizens in check. But police patrols are rare in Oktyabrskaya, partly due to its location on an out-of-the-way peninsula in a bend of the river Svislach. When the first restaurant opened there in 2012, few visitors came but now it ranks among the most fashionable quarters of Minsk.  Such is the growing popularity of that Oktyabrskaya that investors such as Belgazprombank, a subsidiary of state-owned Russian lender Gazprombank, have big plans for the district. Earlier this year the bank purchased part of a factory there and intends to turn it into a gallery, restaurant and theater complex. The manufacturing sector has not entirely abandoned Oktyabrskaya - one  machine-making factory named after the 1917 October Revolution (MZOR) still operates there. Financial difficulties prompted state-owned MZOR to lease or sell some of its facilities to Oktyabrskaya s developers, but the firm still maintains some production with a reduced workforce. Mikhail, who has worked at the factory for 42 years, said he approves of the influx of new  hipster  businesses.  The street has come back to life,  he said. Oktyabrskaya s long-term future ultimately depends on the authorities good favor, cultural analyst Maksim Zhbankov said.  For now they tolerate it. But I can t say that someone won t turn up tomorrow and say they ve decided to tear it all down,  he said.